## Imports

In [1]:
import os
import random

import cv2
import numpy as np
import pandas as pd

import tensorflow as tf
print("Tensorflow version " + tf.__version__)
from tensorflow.keras import layers, models
from keras.callbacks import Callback

Tensorflow version 2.13.0


In [2]:
# %load_ext tensorboard

## Extract dataset from drive

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
!mkdir dataset/

In [5]:
!unzip -q drive/MyDrive/dataset-cnn-trafic/43-classes.zip -d dataset/

## Define paths

In [6]:
path_dataset = "dataset"
path_train_csv = os.path.join(path_dataset, "train.csv")
path_test_csv = os.path.join(path_dataset, "test.csv")
path_train_imgs = os.path.join(path_dataset, "train")
path_test_imgs = os.path.join(path_dataset, "test")

## Load dataframes

In [7]:
df_train_val = pd.read_csv(path_train_csv, sep=";", index_col=0)
df_test = pd.read_csv(path_test_csv, sep=";", index_col=0)

## Split into train and validate datasets

In [8]:
random.seed(10)
train_val_index = np.array(df_train_val.index)
random.shuffle(train_val_index)

ratio = 0.07

df_val = df_train_val.iloc[train_val_index[:int(len(train_val_index)*ratio)]]
df_train = df_train_val.iloc[train_val_index[int(len(train_val_index)*ratio):]]

In [9]:
train_images = df_train["Filename"].apply(lambda f:
 tf.keras.utils.img_to_array(tf.keras.utils.load_img(os.path.join(path_train_imgs,f)))/ 255.0)
train_images = np.concatenate([[arr] for arr in train_images])

In [10]:
val_images = df_val["Filename"].apply(lambda f:
 tf.keras.utils.img_to_array(tf.keras.utils.load_img(os.path.join(path_train_imgs,f)))/ 255.0)
val_images = np.concatenate([[arr] for arr in val_images])

In [11]:
train_labels = df_train["ClassId"].to_numpy()
train_lables = train_labels.reshape(-1,1)

In [12]:
val_labels = df_val["ClassId"].to_numpy()
val_labels = val_labels.reshape(-1,1)

In [13]:
NUM_CLASSES = 43

model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(48, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (5, 5), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(NUM_CLASSES, activation='relu'))
model.add(layers.Dense(NUM_CLASSES))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 48)        38448     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 48)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 32)          38432     
                                                                 
 flatten (Flatten)           (None, 32)                0

In [14]:
class SaveEveryNEpochs(Callback):
    def __init__(self, filepath, n_epochs):
        super(SaveEveryNEpochs, self).__init__()
        self.filepath = filepath
        self.n_epochs = n_epochs

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.n_epochs == 0:
            print("Saving")
            self.model.save(self.filepath.format(epoch=epoch + 1))

In [15]:
my_callbacks = [
    SaveEveryNEpochs(filepath="model.{epoch:02d}.keras", n_epochs=30),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

history = model.fit(train_images, train_labels, epochs=120, batch_size=2048,
                    validation_data=(val_images, val_labels), callbacks=my_callbacks)

Epoch 1/120
 5/18 [=======>......................] - ETA: 0s - loss: 3.7528 - accuracy: 0.0313

18/18 [==============================] - 14s 118ms/step - loss: 3.6616 - accuracy: 0.0508 - val_loss: 3.4650 - val_accuracy: 0.0791
Epoch 2/120
18/18 [==============================] - 1s 68ms/step - loss: 3.1483 - accuracy: 0.2172 - val_loss: 2.7738 - val_accuracy: 0.3017
Epoch 3/120
18/18 [==============================] - 1s 71ms/step - loss: 2.4215 - accuracy: 0.4125 - val_loss: 2.0851 - val_accuracy: 0.4985
Epoch 4/120
18/18 [==============================] - 1s 52ms/step - loss: 1.7621 - accuracy: 0.5662 - val_loss: 1.4894 - val_accuracy: 0.6348
Epoch 5/120
18/18 [==============================] - 1s 53ms/step - loss: 1.2518 - accuracy: 0.6879 - val_loss: 1.0757 - val_accuracy: 0.7456
Epoch 6/120
18/18 [==============================] - 1s 48ms/step - loss: 0.9280 - accuracy: 0.7711 - val_loss: 0.8212 - val_accuracy: 0.8058
Epoch 7/120
18/18 [==============================] - 1s 48ms/step - loss: 0.7289 - accuracy: 0.8302 - val_loss: 0.6653 - val_accuracy: 0.8550
Epoch 8/120
18/1

In [16]:
model.save('my_model.keras')

In [17]:
test_images = df_test["Filename"].apply(lambda f:
 [tf.keras.utils.img_to_array(tf.keras.utils.load_img(os.path.join(path_test_imgs, f)))/ 255.0])
test_images = np.concatenate(test_images)

test_labels = df_test["ClassId"].to_numpy()
test_labels = test_labels.reshape(-1,1)

In [18]:
model.evaluate(test_images, test_labels)

395/395 [==============================] - 2s 4ms/step - loss: 0.6826 - accuracy: 0.9392


[0.6826120615005493, 0.9391924142837524]

In [19]:
model.predict(np.expand_dims(test_images[-1], axis=0)).argmax(axis=-1)

1/1 [==============================] - 0s 195ms/step


array([10])